# Neural Network tester
- Load a pre-trained NN and test it against 1296 passwords
- Save a CSV with the hystory and cuts (it could be used by *matches_analyzer.ipynb*)

**TODO**
- Salvare statistiche cuts

-----------------------

In [110]:
PRINT_INFO = True

NETWORKS_FAMILY_PATH = "../networks/Trained/hopeful"

---------------------------------------------
## Load the networks

In [111]:
import tensorflow as tf
import pandas as pd
import json
import os

In [112]:
with open(INFOS_PATH[0]) as f:
    nn_info = json.load(f)
    print("Info network 0:"+json.dumps(nn_info, indent=4, sort_keys=True))

Info network 0:{
    "ALPHA": 0.001,
    "ATTENTION_SIZE": 256,
    "BATCH_EVAL": 32,
    "BATCH_TRAIN": 32,
    "BW_FORGET_BIAS": 1,
    "CSV_PATH_EVAL": "../database/dbML/hopeful/JOINED_hopeful_2019-07-01_0.7_eval.csv",
    "CSV_PATH_TRAIN": "../database/dbML/hopeful/JOINED_hopeful_2019-07-01_0.7_train.csv",
    "DROP_REPRESENTATION": 0.3,
    "EPOCHS": 10,
    "EVAL_STEPS": null,
    "FW_FORGET_BIAS": 1,
    "GOAL": "goalReduced",
    "GOAL_POS": 0,
    "INPUT_ENCODING": "Peg: Hot encoding, Tips: One Hot Encoding",
    "LSTM_SIZE": 256,
    "MAXOUT_SIZE": 256,
    "MODEL_DIR_PATH": "model_dir_647_0",
    "MULTI_THREADING": true,
    "NUM_EVALS": 10,
    "SHUFFLE_EVAL": false,
    "_NETWORK_TYPE": "BiLSTM_AttentionSystem",
    "loss_fn": "mean_pairwise_squared_error",
    "train_duration": "4:05:37"
}


In [113]:
def load_model(model_path):
    predictor_fn = tf.contrib.predictor.from_saved_model(
                            export_dir = model_path,
                            signature_def_key="prediction")
    return predictor_fn

models = []
for path in MODELS_PATH:
    model = load_model(path)
    models.append(model)

INFO:tensorflow:Restoring parameters from ../networks/Trained/hopeful/model_dir_647_0/export/predict/1562032796/variables/variables
INFO:tensorflow:Restoring parameters from ../networks/Trained/hopeful/model_dir_341_1/export/predict/1562047367/variables/variables
INFO:tensorflow:Restoring parameters from ../networks/Trained/hopeful/model_dir_589_2/export/predict/1562062221/variables/variables
INFO:tensorflow:Restoring parameters from ../networks/Trained/hopeful/model_dir_496_3/export/predict/1562076759/variables/variables


------------------
## Test the network

In [114]:
from itertools import product
from string import ascii_uppercase
FEATURES_NAME = ['Guess 1', 'Guess 2', 'Guess 3', 'Guess 4', 'Guess 5',
          'Guess 6', 'Guess 7', 'Guess 8', 'Guess 9', 'Guess 10']

TARGET_NAME = 'PASSWORD'

HEADER = FEATURES_NAME + [TARGET_NAME]

HOT_ENCODING = {
    "100000" : 'A',
    "010000" : 'B',
    "001000" : 'C',
    "000100" : 'D',
    "000010" : 'E',
    "000001" : 'F',
}

MATCH_LOST_PAD = 'XXXX'

# Classic Mastermind rules
PSW_CARDINALITY = 6
PSW_LEN = 4
PSW_POOL_COMPLETE = [''.join(p) for p in product(ascii_uppercase[0:PSW_CARDINALITY], repeat=PSW_LEN)]

# Analysis persistance
def save_to_csv(matches, tail_name):
    df = pd.DataFrame(data=matches, columns=HEADER)
    with open('./matchesPlayed/{}_{}.csv'.format(NETWORKS_NAME, tail_name),'w') as f:
        df.to_csv(f, sep=',', encoding='utf-8',index=False, header=True, float_format="%.0f")

In [115]:
def change_char(s, p, r):
    return s[:p]+r+s[p+1:]

def hints_calculator(psw_goal, psw_guess):
    black_pegs = len([goal_element for goal_element, guess_element in zip(psw_goal, psw_guess) if goal_element == guess_element])
    white_pegs = sum([min(psw_goal.count(element), psw_guess.count(element)) for element in set('ABCDEF')]) - black_pegs
    return black_pegs, white_pegs

def get_clean_match():
    match = {}
    cuts = {}
    for guess in FEATURES_NAME:
        match[guess] = ["<pad>"]
        cuts[guess] = 0
    match[TARGET_NAME] = ["----"]
    cuts[TARGET_NAME] = "----"
    return match, cuts

def make_a_guess(match, psw):
    for idx, model in enumerate(models):
        prediction = model(match)['Prediction'][0]
        prediction = "".join(str(int(digit)) for digit in prediction)
        prediction = HOT_ENCODING[prediction]
        match[TARGET_NAME][0] = change_char(match[TARGET_NAME][0], idx, prediction)
    blk_peg, wht_peg = hints_calculator(psw, match[TARGET_NAME][0])
    guess = "{}{}{}".format(match[TARGET_NAME][0], blk_peg, wht_peg)
    match[TARGET_NAME][0] = '----'
    won = blk_peg == 4
    return guess, won, (blk_peg, wht_peg)

def guess_pool_pruning(psw_pool, next_guess, hints):
    to_remove = []
    for psw in psw_pool:
        if hints != hints_calculator(psw, next_guess):
            to_remove.append(psw)
    len_psw_pool = len(psw_pool)
    psw_pool.difference_update(to_remove)
    return int(len_psw_pool - len(psw_pool))

In [116]:
from pprint import pprint

victories = 0
matches = []
cuts_hist = []

for PSW in PSW_POOL_COMPLETE:
    won = False
    psw_pool = set(PSW_POOL_COMPLETE.copy())
    match, cuts = get_clean_match()
    for guess_pos in FEATURES_NAME:
        guess, is_psw, pegs = make_a_guess(match, PSW)
        match[guess_pos] = [guess]
        cuts[guess_pos] = guess_pool_pruning(psw_pool, guess[:-2], pegs)
        if is_psw:
            won = True
            break
            
    match = {guess_pos: guess[0] for guess_pos, guess in match.items()}
    match[TARGET_NAME] = PSW
    cuts[TARGET_NAME] = PSW
    if won:
        victories = victories + 1
    else:
        match['Guess 10'] = MATCH_LOST_PAD
        print("Psw {} not guessed, match played:\n".format(PSW))
        pprint(match)
        print('\n\n')
    matches.append(match)
    cuts_hist.append(cuts)

Psw AAAB not guessed, match played:

{'Guess 1': 'AEFE10',
 'Guess 10': 'XXXX',
 'Guess 2': 'ABBD11',
 'Guess 3': 'BEBC01',
 'Guess 4': 'DDAE10',
 'Guess 5': 'DBFF01',
 'Guess 6': 'CCAF10',
 'Guess 7': 'ADAB30',
 'Guess 8': 'ADAB30',
 'Guess 9': 'ADAB30',
 'PASSWORD': 'AAAB'}



Psw AACC not guessed, match played:

{'Guess 1': 'AEFE10',
 'Guess 10': 'XXXX',
 'Guess 2': 'ABBD10',
 'Guess 3': 'CEBC11',
 'Guess 4': 'CCFD02',
 'Guess 5': 'DFDC10',
 'Guess 6': 'AECC30',
 'Guess 7': 'AEAC21',
 'Guess 8': 'AECF20',
 'Guess 9': 'AEAC21',
 'PASSWORD': 'AACC'}



Psw ABAE not guessed, match played:

{'Guess 1': 'AEFE20',
 'Guess 10': 'XXXX',
 'Guess 2': 'AEBB12',
 'Guess 3': 'BEAC12',
 'Guess 4': 'ABDE30',
 'Guess 5': 'ABBE30',
 'Guess 6': 'ABBE30',
 'Guess 7': 'ABFE30',
 'Guess 8': 'ABCE30',
 'Guess 9': 'ABEE30',
 'PASSWORD': 'ABAE'}



Psw ABEF not guessed, match played:

{'Guess 1': 'AEFE12',
 'Guess 10': 'XXXX',
 'Guess 2': 'EEAB03',
 'Guess 3': 'BAEF22',
 'Guess 4': 'FAEB13',
 'Guess 5': 'B

In [117]:
save_to_csv(matches, 'play')
save_to_csv(cuts_hist, 'play_cuts')

In [118]:
print("Victories: {} / {}".format(victories, len(PSW_POOL_COMPLETE)))
print("Ratio: {}".format(victories/len(PSW_POOL_COMPLETE)))

Victories: 1181 / 1296
Ratio: 0.9112654320987654
